<a href="https://colab.research.google.com/github/Tobias-Rom3ro/Pipeline-DirtyIris/blob/main/Iris_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌸 Pipeline de Entrenamiento de ML con Dataset Iris.

## 1. Importación de librerías.

In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
import joblib

## 2. Funciones auxiliares.

In [72]:
def load_data(file_path):
    """Solo carga los datos del CSV"""
    return pd.read_csv(file_path)

In [73]:
def prepare_target_variable(df):
    """Prepara la variable target aplicando la misma limpieza"""
    Species_mapping = {
        'setosa': 'setosa',
        'iris-setosa': 'setosa',
        'versicolor': 'versicolor',
        'iris-versicolor': 'versicolor',
        'virginica': 'virginica',
        'iris-virginica': 'virginica'
    }

    # Limpiar eSpecies para y
    y_clean = df['Species'].str.lower().str.strip().map(Species_mapping)

    # Codificar
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_clean)

    return y_encoded, label_encoder

## 3. Transformers personalizados.

In [74]:
class DataExplorer(BaseEstimator, TransformerMixin):
    """Transformer que explora y muestra información de los datos"""

    def __init__(self, verbose=True):
        self.verbose = verbose
        self.original_shape = None
        self.Species_counts = None

    def fit(self, X, y=None):
        if self.verbose:
            print("="*50)
            print("PASO 1: EXPLORACIÓN DE DATOS")
            print("="*50)
            print(f"Forma del dataset: {X.shape}")
            print("\nPrimeras 5 filas:")
            print(X.head())

            if 'Species' in X.columns:
                print("\nValores únicos en Species (antes de limpiar):")
                print(X['Species'].value_counts())

            print("\nValores nulos:")
            print(X.isnull().sum())

        self.original_shape = X.shape
        return self

    def transform(self, X):
        return X

In [75]:
class SpeciesCleaner(BaseEstimator, TransformerMixin):
    """Transformer que limpia automáticamente los nombres de eSpecies"""

    def __init__(self, verbose=True):
        self.verbose = verbose
        self.Species_mapping = {
            'setosa': 'setosa',
            'iris-setosa': 'setosa',
            'versicolor': 'versicolor',
            'iris-versicolor': 'versicolor',
            'virginica': 'virginica',
            'iris-virginica': 'virginica'
        }

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.verbose:
            print("\n" + "="*50)
            print("PASO 2: LIMPIEZA DE ESpecies")
            print("="*50)

        df = X.copy()

        if 'Species' in df.columns:
            # Mostrar antes de limpiar
            if self.verbose:
                print("ESpecies antes de limpiar:")
                print(df['Species'].value_counts())

            # Limpiar eSpecies
            df['Species'] = df['Species'].str.lower().str.strip()
            df['Species'] = df['Species'].map(self.Species_mapping)

            # Mostrar después de limpiar
            if self.verbose:
                print("\nESpecies después de limpiar:")
                print(df['Species'].value_counts())

        return df

In [76]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """Transformer que selecciona solo las features numéricas"""

    def __init__(self, verbose=True):
        self.verbose = verbose
        self.feature_columns = ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.verbose:
            print("\n" + "="*50)
            print("PASO 3: SELECCIÓN DE FEATURES")
            print("="*50)
            print(f"Columnas originales: {list(X.columns)}")
            print(f"Features seleccionadas: {self.feature_columns}")

        df = X.copy()
        selected_features = df[self.feature_columns]

        if self.verbose:
            print(f"Forma después de selección: {selected_features.shape}")

        return selected_features

## 4. Creación del pipeline completo.

In [77]:
def create_complete_pipeline():
    # Pipeline de preprocesamiento para features numéricas
    feature_cols = ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']

    numeric_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # ColumnTransformer para manejar las features
    preprocessor = ColumnTransformer([
        ('numeric', numeric_pipeline, feature_columns)
    ], remainder='drop')

    complete_pipeline = Pipeline([
        ('explorer', DataExplorer(verbose=True)),
        ('cleaner', SpeciesCleaner(verbose=True)),
        ('selector', FeatureSelector(verbose=True)),
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])

    return complete_pipeline

## 5. Entrenamiento, evaluación y guardado del pipeline.

In [78]:
def train_iris_pipeline(file_path, test_size=0.2, random_state=42):

    print("="*60)
    print("ENTRENAMIENTO CON PIPELINE")
    print("="*60)

    # 1. Cargar datos
    print("Cargando datos del archivo:", file_path)
    df = load_data(file_path)
    print("Columnas después de cargar:", df.columns.tolist()) # Add this line to check columns


    # 2. Aplicar DataExplorer y SpeciesCleaner antes de preparar target
    print("\nAplicando DataExplorer y SpeciesCleaner...")
    data_explorer = DataExplorer(verbose=True)
    df_explored = data_explorer.transform(df) # Just for exploration output
    Species_cleaner = SpeciesCleaner(verbose=True)
    df_cleaned = Species_cleaner.transform(df_explored)
    print("Columnas después de limpiar eSpecies:", df_cleaned.columns.tolist()) # Add this line to check columns


    # 3. Preparar target
    print("\nPreparando variable target...")
    y, label_encoder = prepare_target_variable(df_cleaned)

    # 4. Define features (X) after cleaning Species but before selecting numeric ones
    X = df_cleaned.drop('Species', axis=1, errors='ignore') # Drop Species from features for pipeline input


    # 5. División de datos
    print(f"\nDividiendo datos ({int((1-test_size)*100)}% entrenamiento, {int(test_size*100)}% prueba)...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    print(f"Entrenamiento: {X_train.shape[0]} muestras")
    print(f"Prueba: {X_test.shape[0]} muestras")

    # 6. Crear pipeline (Starts with FeatureSelector for X_train)
    print("\nCreando pipeline de entrenamiento...")
    feature_columns = ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']

    numeric_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer([
        ('numeric', numeric_pipeline, feature_columns)
    ], remainder='drop')


    complete_pipeline = Pipeline([
        ('selector', FeatureSelector(verbose=True)), # Select features from X
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])


    # 7. ENTRENAR
    print("\n" + "="*60)
    print("ENTRENANDO PIPELINE")
    print("="*60)
    # Fit the pipeline on the training features and target
    complete_pipeline.fit(X_train, y_train)

    # 8. EVALUAR
    print("\n" + "="*60)
    print("EVALUANDO PIPELINE")
    print("="*60)

    y_pred = complete_pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
    print("\nMatriz de confusión:")
    print(confusion_matrix(y_test, y_pred))

    # 9. Guardar pipeline
    print("\n" + "="*60)
    print("GUARDANDO PIPELINE")
    print("="*60)
    joblib.dump(complete_pipeline, 'complete_iris_pipeline.pkl')
    joblib.dump(label_encoder, 'label_encoder.pkl')
    print("Pipeline guardado en: complete_iris_pipeline.pkl")
    print("Label encoder guardado en: label_encoder.pkl")

    return complete_pipeline, label_encoder

## 6. Ejecución principal.

In [79]:
if __name__ == "__main__":
    file_path = 'iris_dirty.csv'
    trained_pipeline, label_encoder = train_iris_pipeline(file_path)

ENTRENAMIENTO CON PIPELINE
Cargando datos del archivo: iris_dirty.csv
Columnas después de cargar: ['Unnamed: 0', 'Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']

Aplicando DataExplorer y SpeciesCleaner...

PASO 2: LIMPIEZA DE ESpecies
ESpecies antes de limpiar:
Species
virginica     49
setosa        48
versicolor    48
Setosa         1
SETOSA         1
Versicolor     1
VERSICOLOR     1
VIRGINICA      1
Name: count, dtype: int64

ESpecies después de limpiar:
Species
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64
Columnas después de limpiar eSpecies: ['Unnamed: 0', 'Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']

Preparando variable target...

Dividiendo datos (80% entrenamiento, 20% prueba)...
Entrenamiento: 120 muestras
Prueba: 30 muestras

Creando pipeline de entrenamiento...

ENTRENANDO PIPELINE

PASO 3: SELECCIÓN DE FEATURES
Columnas originales: ['Unnamed: 0', 'Sepal.Length', 'Sepal.Width', 'Petal.Length

## 7. Usar el pipeline entrenado.

In [80]:
def load_pipeline():
    """Carga el pipeline entrenado"""
    pipeline = joblib.load('complete_iris_pipeline.pkl')
    label_encoder = joblib.load('label_encoder.pkl')
    return pipeline, label_encoder

In [81]:
def predict_new_samples(pipeline, label_encoder, new_data):
    """Usa el pipeline para predecir nuevas muestras"""
    predictions = pipeline.predict(new_data)
    probabilities = pipeline.predict_proba(new_data)
    Species_names = label_encoder.inverse_transform(predictions)

    return Species_names, probabilities

## 8. Prueba con datos ficticios.

In [85]:
try:
    # Cargar pipeline entrenado
    pipeline, label_encoder = load_pipeline()

    # Crear datos ficticios
    test_data = pd.DataFrame({
        'Sepal.Length': [5.1, 6.2, 7.3, 4.8, 5.8],
        'Sepal.Width': [3.5, 2.2, 2.9, 3.0, 2.7],
        'Petal.Length': [1.4, 4.5, 6.3, 1.4, 5.1],
        'Petal.Width': [0.2, 1.5, 1.8, 0.1, 1.9],
        'Species': ['SETOSA', 'iris-versicolor', 'VIRGINICA', 'setosa', 'unknown']  # Datos sucios a propósito
    })

    print("Datos de prueba (sucios):")
    print(test_data)

    Species_predictions, probabilities = predict_new_samples(pipeline, label_encoder, test_data)

    print("\nRESULTADOS:")
    print("-"*50)
    for i, (pred, prob) in enumerate(zip(Species_predictions, probabilities)):
        max_prob = max(prob)
        print(f"Muestra {i+1}: {pred} (confianza: {max_prob:.4f})")

        for Species, probability in zip(label_encoder.classes_, prob):
            print(f"  {Species}: {probability:.4f}")
        print()

    print("Pipeline funcionando perfectamente")

except FileNotFoundError:
    print("Pipeline no encontrado.")
except Exception as e:
    print(f"Error: {e}")


Datos de prueba (sucios):
   Sepal.Length  Sepal.Width  Petal.Length  Petal.Width          Species
0           5.1          3.5           1.4          0.2           SETOSA
1           6.2          2.2           4.5          1.5  iris-versicolor
2           7.3          2.9           6.3          1.8        VIRGINICA
3           4.8          3.0           1.4          0.1           setosa
4           5.8          2.7           5.1          1.9          unknown

PASO 3: SELECCIÓN DE FEATURES
Columnas originales: ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']
Features seleccionadas: ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']
Forma después de selección: (5, 4)

PASO 3: SELECCIÓN DE FEATURES
Columnas originales: ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']
Features seleccionadas: ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']
Forma después de selección: (5, 4)

RESULTADOS:
---------------------------